# 04 - Generar Reportes

Este notebook genera reportes completos del análisis paramétrico usando la clase `ReportGenerator`.

## Objetivos:
- Cargar todos los resultados de análisis
- Generar reportes de desplazamientos
- Generar reportes modales
- Crear reporte completo con visualizaciones

In [ ]:
import sys
sys.path.append('../src')

from report_generator import ReportGenerator
import os
import pandas as pd

## 1. Inicializar el Generador de Reportes

In [ ]:
# Inicializar el generador de reportes
reporter = ReportGenerator(
    results_dir="../results",
    reports_dir="../reports"
)

print("Generador de reportes inicializado")
print(f"Directorio de resultados: {reporter.results_dir}")
print(f"Directorio de reportes: {reporter.reports_dir}")

## 2. Cargar Todos los Resultados

In [ ]:
# Cargar todos los resultados disponibles
all_results = reporter.load_all_results()

print(f"Resultados cargados: {len(all_results)}")

if all_results:
    print("\nEjemplos de modelos analizados:")
    for i, result in enumerate(all_results[:5]):
        print(f"{i+1}. {result['model_name']}")
        print(f"   L/B={result['model_parameters']['L_B_ratio']}, ")
        print(f"   nx={result['model_parameters']['nx']}, ny={result['model_parameters']['ny']}")
        print(f"   Estático: {'✓' if result['static_analysis']['success'] else '✗'}")
        print(f"   Modal: {'✓' if result['modal_analysis']['success'] else '✗'}")
    
    if len(all_results) > 5:
        print(f"   ... y {len(all_results) - 5} más")
else:
    print("No hay resultados disponibles. Ejecuta primero el notebook 03_analisis_parametrico_completo.ipynb")

## 3. Crear DataFrame Resumen

In [ ]:
if all_results:
    # Crear DataFrame resumen
    df = reporter.create_summary_dataframe(all_results)
    
    print("=== DATAFRAME RESUMEN ===")
    print(f"Dimensiones: {df.shape}")
    print(f"Columnas: {list(df.columns)}")
    
    print("\nPrimeras 5 filas:")
    print(df.head())
    
    print("\nEstadísticas básicas:")
    print(f"Análisis estáticos exitosos: {df['static_success'].sum()}/{len(df)}")
    print(f"Análisis modales exitosos: {df['modal_success'].sum()}/{len(df)}")
    
    # Mostrar rangos de parámetros
    print(f"\nRangos de parámetros:")
    print(f"L/B ratios: {sorted(df['L_B_ratio'].unique())}")
    print(f"nx valores: {sorted(df['nx'].unique())}")
    print(f"ny valores: {sorted(df['ny'].unique())}")

## 4. Generar Reporte de Desplazamientos

In [ ]:
if all_results:
    print("Generando reporte de desplazamientos...")
    
    displacement_report = reporter.generate_displacement_report(all_results)
    
    if displacement_report:
        print("\n=== REPORTE DE DESPLAZAMIENTOS ===")
        print(f"Gráficas generadas: {len(displacement_report['plots'])}")
        for plot_name, plot_path in displacement_report['plots'].items():
            print(f"  - {plot_name}: {plot_path}")
        
        print(f"\nArchivo CSV: {displacement_report['csv_file']}")
        
        print("\nEstadísticas:")
        stats = displacement_report['summary_stats']
        print(f"  - Total de modelos: {stats['total_models']}")
        print(f"  - Modelos exitosos: {stats['successful_models']}")
        print(f"  - Desplazamiento máximo: {stats['max_displacement']:.6f} m")
        print(f"  - Desplazamiento mínimo: {stats['min_displacement']:.6f} m")
        print(f"  - Desplazamiento promedio: {stats['mean_displacement']:.6f} m")
    else:
        print("No se pudo generar el reporte de desplazamientos")

## 5. Generar Reporte Modal

In [ ]:
if all_results:
    print("\nGenerando reporte modal...")
    
    modal_report = reporter.generate_modal_report(all_results)
    
    if modal_report:
        print("\n=== REPORTE MODAL ===")
        print(f"Gráficas generadas: {len(modal_report['plots'])}")
        for plot_name, plot_path in modal_report['plots'].items():
            print(f"  - {plot_name}: {plot_path}")
        
        print(f"\nArchivo CSV: {modal_report['csv_file']}")
        
        print("\nEstadísticas:")
        stats = modal_report['summary_stats']
        print(f"  - Total de modelos: {stats['total_models']}")
        print(f"  - Modelos exitosos: {stats['successful_models']}")
        print(f"  - Período máximo: {stats['max_period']:.4f} s")
        print(f"  - Período mínimo: {stats['min_period']:.4f} s")
        print(f"  - Período promedio: {stats['mean_period']:.4f} s")
        print(f"  - Frecuencia máxima: {stats['max_frequency']:.4f} Hz")
        print(f"  - Frecuencia mínima: {stats['min_frequency']:.4f} Hz")
        print(f"  - Frecuencia promedio: {stats['mean_frequency']:.4f} Hz")
    else:
        print("No se pudo generar el reporte modal")

## 6. Generar Reporte Completo

In [ ]:
if all_results:
    print("\nGenerando reporte completo...")
    
    comprehensive_report = reporter.generate_comprehensive_report(all_results)
    
    print("\n=== REPORTE COMPLETO ===")
    print(f"Reporte HTML: {comprehensive_report['html_report']}")
    print(f"Resumen ejecutivo: {comprehensive_report['summary_file']}")
    
    print("\nReportes individuales incluidos:")
    print(f"  - Reporte de desplazamientos: {len(comprehensive_report['displacement_report']['plots'])} gráficas")
    print(f"  - Reporte modal: {len(comprehensive_report['modal_report']['plots'])} gráficas")

## 7. Verificar Archivos Generados

In [ ]:
# Verificar archivos de reportes generados
if os.path.exists(reporter.reports_dir):
    report_files = os.listdir(reporter.reports_dir)
    
    print("\n=== ARCHIVOS DE REPORTES GENERADOS ===")
    print(f"Total de archivos: {len(report_files)}")
    
    # Categorizar archivos
    png_files = [f for f in report_files if f.endswith('.png')]
    html_files = [f for f in report_files if f.endswith('.html')]
    csv_files = [f for f in report_files if f.endswith('.csv')]
    txt_files = [f for f in report_files if f.endswith('.txt')]
    
    print(f"\nGráficas PNG: {len(png_files)}")
    for file in png_files:
        print(f"  - {file}")
    
    print(f"\nArchivos HTML: {len(html_files)}")
    for file in html_files:
        print(f"  - {file}")
    
    print(f"\nArchivos CSV: {len(csv_files)}")
    for file in csv_files:
        print(f"  - {file}")
    
    print(f"\nArchivos TXT: {len(txt_files)}")
    for file in txt_files:
        print(f"  - {file}")
else:
    print("Directorio de reportes no existe")

## 8. Mostrar Resumen Ejecutivo

In [ ]:
# Mostrar contenido del resumen ejecutivo
summary_file = os.path.join(reporter.reports_dir, 'executive_summary.txt')
if os.path.exists(summary_file):
    print("\n=== RESUMEN EJECUTIVO ===")
    with open(summary_file, 'r', encoding='utf-8') as f:
        content = f.read()
        print(content)
else:
    print("Archivo de resumen ejecutivo no encontrado")

## 9. Resumen Final

In [ ]:
print("\n=== RESUMEN FINAL ===")
print(f"✓ Resultados analizados: {len(all_results)}")
print(f"✓ Reportes generados: {len(report_files) if 'report_files' in locals() else 0}")
print(f"✓ Gráficas creadas: {len(png_files) if 'png_files' in locals() else 0}")
print(f"✓ Archivos HTML: {len(html_files) if 'html_files' in locals() else 0}")
print(f"✓ Archivos CSV: {len(csv_files) if 'csv_files' in locals() else 0}")

print("\n✅ Generación de reportes completada exitosamente!")
print("\nArchivos importantes generados:")
print("1. comprehensive_report.html - Reporte completo en HTML")
print("2. executive_summary.txt - Resumen ejecutivo")
print("3. displacement_summary.csv - Datos de desplazamientos")
print("4. modal_summary.csv - Datos modales")
print("5. Gráficas PNG - Visualizaciones de resultados")
print("6. Gráficas HTML 3D - Visualizaciones interactivas")

print("\nPróximos pasos:")
print("1. Abrir comprehensive_report.html en tu navegador")
print("2. Revisar las gráficas PNG generadas")
print("3. Analizar los archivos CSV con los datos")
print("4. Revisar el resumen ejecutivo")